# ANN Regression for Customer Estimated Salary Prediction

### This Jupyter Notebook demonstrates how to build and train an Artificial Neural Network (ANN) for a regression task.
### Specifically, we will predict the 'EstimatedSalary' of a customer based on other features in the `Churn_Modelling.csv` dataset.
### This serves as an example of applying ANN to a continuous target variable.

## Key Steps Covered:

- 1.  **Data Loading and Initial Preprocessing**: Reading the dataset and dropping irrelevant columns.
- 2.  **Categorical Feature Encoding**: Applying `LabelEncoder` and `OneHotEncoder` to transform categorical features ('Gender' and 'Geography') into numerical representations.
- 3.  **Data Splitting (Features and Target)**: Defining 'EstimatedSalary' as the dependent variable (target `y`) and all other columns as independent features (`X`).
- 4.  **Train-Test Split**: Dividing the data into training and testing sets.
- 5.  **Feature Scaling**: Standardizing numerical features using `StandardScaler` to ensure optimal ANN performance.
- 6.  **Saving Preprocessing Objects**: Persisting the trained encoders and scaler using `pickle` for consistent future use.
- 7.  **ANN Model Building (Regression)**: Constructing a sequential ANN with dense layers. The key difference from classification is the output layer, which does not have an activation function (or uses a linear activation) and outputs a single continuous value.
- 8.  **Model Compilation**: Configuring the model with an optimizer (`adam`) and a regression-specific loss function (`mean_absolute_error`) and metric (`mae`).
- 9.  **Callback Setup**: Implementing `EarlyStopping` to prevent overfitting and `TensorBoard` for visualizing training progress and metrics.
- 10. **Model Training**: Fitting the ANN model to the training data.
- 11. **TensorBoard Visualization**: Launching TensorBoard to observe training and validation curves.
- 12. **Model Evaluation**: Assessing the model's performance on unseen test data using Mean Absolute Error (MAE).
- 13. **Model Saving**: Saving the trained regression ANN model for later deployment.

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
# Load the dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data
# Drop irrelevant columns: RowNumber, CustomerId, Surname are unique identifiers and not predictive features.
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
# Encode categorical variables
# Initialize LabelEncoder for the 'Gender' column
label_encoder_gender = LabelEncoder()
# Fit and transform the 'Gender' column (e.g., 'Male' -> 0, 'Female' -> 1)
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
# One-hot encode 'Geography'
# Initialize OneHotEncoder with handle_unknown='ignore' to gracefully handle new, unseen categories during prediction
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
# Fit and transform the 'Geography' column. It expects a 2D array, so pass `data[['Geography']]`.
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
# Create a DataFrame from the one-hot encoded array, assigning appropriate column names
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
# Combine one-hot encoded columns with original data
# Drop the original 'Geography' column as it's now encoded
# Concatenate the modified 'data' DataFrame and the 'geo_encoded_df' column-wise
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
# Split the data into features (X) and target (y)
# X will contain all features except 'EstimatedSalary'
X = data.drop('EstimatedSalary', axis=1)
# y will contain the 'EstimatedSalary' column, which is our continuous target variable
y = data['EstimatedSalary']

In [9]:
## Split the data into training and testing sets
# Split X and y into training and testing sets (80% train, 20% test)
# random_state ensures reproducibility of the split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
## Scale these features
# Initialize StandardScaler for numerical feature scaling
scaler=StandardScaler()
# Fit the scaler on the training data and transform it. This learns mean and std from X_train.
X_train=scaler.fit_transform(X_train)
# Transform the test data using the *same* scaler fitted on the training data
X_test=scaler.transform(X_test)

In [11]:
# Save the encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

#### ANN Regression Problem statement

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Matplotlib is building the font cache; this may take a moment.


In [13]:
# Build the Sequential ANN model for regression
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

## compile the model
# Compile the model with the Adam optimizer and Mean Absolute Error (MAE) as the loss function and metric
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

# Display the model summary, showing layer types, output shapes, and parameter counts
model.summary()

/Users/pro/python-learning/Machine Learning/3-ann-classification-project/ann-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
# Define the log directory for TensorBoard, creating a unique timestamped folder for each run
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Create a TensorBoard callback instance, logging histograms of weights and biases
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
# Set up Early Stopping
# Create an EarlyStopping callback instance to monitor 'val_loss'
# patience=10: Training will stop if 'val_loss' doesn't improve for 10 consecutive epochs
# restore_best_weights=True: Model weights from the epoch with the best 'val_loss' will be restored
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [16]:
# Train the model
# Fit the model to the training data

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 100264.0469 - mae: 100264.0469 - val_loss: 98513.2344 - val_mae: 98513.2344
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - loss: 100796.8125 - mae: 100796.8125 - val_loss: 96887.2500 - val_mae: 96887.2500
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - loss: 97693.0469 - mae: 97693.0469 - val_loss: 92724.0391 - val_mae: 92724.0391
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - loss: 93674.2734 - mae: 93674.2734 - val_loss: 85831.9062 - val_mae: 85831.9062
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - loss: 85722.3047 - mae: 85722.3047 - val_loss: 77092.7188 - val_mae: 77092.7188
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - loss: 76848.4062 - mae: 76848.4062 - val_loss: 68081.0547 - val_mae: 68081.0547
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 66989.8047 - mae: 66989.8047 - val_loss: 60369.7188 - val_mae: 60369.7188
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━

In [17]:
# Load TensorBoard Extension (Jupyter magic command)
%load_ext tensorboard

In [19]:
# Launch TensorBoard to visualize the training logs from the specified directory
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 40425), started 0:00:04 ago. (Use '!kill 40425' to kill it.)

In [20]:
## Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - loss: 51115.2891 - mae: 51115.2891
Test MAE : 50330.765625


In [21]:
model.save('regression_model.h5')